In [1]:
pip install PuLP

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\jknechtel\Miniconda3\python.exe -m pip install --upgrade pip' command.


## Let's try to define the problem mathematically

$$\max_{w \in [0,1]} \sum_{i=0}^{n} w_{i} s_{i} $$

s.t.
$$ \sum_{i=1}^{n} w_{i} RF_{i} \leq WARFTest $$
$$ \sum_{i=1}^{n} w_{i} RR_{i} \geq WARRTest $$
$$ \sum_{i=1}^{n} w_{i} = TotalTargetPar $$
$$ abs(\sum_{i=1}^{n} w_{i} ParBL_{i}) \leq ParBLLim $$
$$ f(w_{i}) \geq DiversityTest $$

Where, $w$ is the par mount of each loan (Par No Default typically).  There might be other constraints such as WAPP, Percent C-, Percent 2ndLien, etc, but they are straightforward like above

$f(\boldsymbol{w})$ is piecewise linear.  Need to determine a way to write it in a way the optimizer can handle.   

$$EUScore_{i} = \min(\frac{Par_{i}}{\overline{Par_{i}}},1)$$
The sector contributions is the trickier part. 
$$AIEUS_{j} = \sum_{i \in sector_{j}} EUScore_{i}$$   
$$AIEUS_{j} \Rightarrow IDS_{j}$$   
This is where we could estimate the functional form. The mapping
looks similar to the sqrt() but with some shifting/scaling.
$$ IDX_{j} = \hat{f}(AIEUS_{j})$$

$$ DiversityScore = \sum_{j} IDS_{j}$$

potentially,
$$ DiversityScore = \sum_{j} \hat{f} [\sum_{i \in sector_{j}}\min(\frac{Par_{i}}{\overline{Par_{i}}},1)] $$

(In the above $Par_{i} = w_{i}$, they are the same, but I wrote it more explicit with $Par$.)




In [2]:
"""
The Simplified Whiskas Model Python Formulation for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

# Import PuLP modeler functions
from pulp import *

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem",LpMinimize)


# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000}

# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0)

# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"

# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print ("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print (v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print ("Total Cost of Ingredients per can = ", value(prob.objective))

C:\Users\jknechtel\Miniconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0
Total Cost of Ingredients per can =  0.52


In [3]:
ingredient_vars

{'CHICKEN': Ingr_CHICKEN,
 'BEEF': Ingr_BEEF,
 'MUTTON': Ingr_MUTTON,
 'RICE': Ingr_RICE,
 'WHEAT': Ingr_WHEAT,
 'GEL': Ingr_GEL}

In [ ]:
def knapSack(W, wt, val, n): 
    K = [[0 for x in range(W + 1)] for x in range(n + 1)] 
   
    # Build table K[][] in bottom up manner 
    for i in range(n + 1): 
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif wt[i-1] <= w: 
                K[i][w] = max(val[i-1] 
                          + K[i-1][w-wt[i-1]],   
                              K[i-1][w]) 
            else: 
                K[i][w] = K[i-1][w] 
   
    return K[n][W] 
   
   
# Driver code 
val = [60, 100, 120] 
wt = [10, 20, 30] 
W = 400000000
n = len(val) 
print(knapSack(W, wt, val, n)) 